# Python Course - Chapter 4 : First numerical algorithms

## 1) Introductory example : solving a second degree equation

This first introductory example will illustrate that testing an equality of `float` variables is complicated. Since a lot of approximations are made when calculating using `float` variables, the condition `x == y` doesn't make much sense and will generally always be false : we prefer testing the condition that `x` and `y` are "close from each other" in a sense we will precise later.<br>
<br>
For now, we will illustrate this issue concerning equality of floats by trying to write a function that solves a second degree equation of the form $$ax^2 + bx + c = 0.$$<br>
Here is a first naive solution of the problem :

In [36]:
from numpy import sqrt

def solve_naive(a, b, c):
    '''
    Parameters : floats a, b, c
    Result : solution(s) of the equation ax^2 + bx + c = 0
    '''
    delta = b**2 - 4*a*c
    if delta < 0:
        print('No real solutions')
    elif delta > 0:
        x1, x2 = (-b-sqrt(delta))/(2*a), (-b+sqrt(delta))/(2*a)
        print('Two solutions :', x1, 'and', x2)
    else:
        x = -b/(2*a)
        print('One solution :', x)

solve_naive(0.01, 0.2, 1)
solve_naive(0.011025, 0.21, 1)

Two solutions : -10.000000131708903 and -9.999999868291098
No real solutions


As you can see, this algorithms concludes that $0.01x^2 + 0.2x + 1 = 0$ has two real solutions and that the equation $0.011025x^2 + 0.21x + 1$ has none.<br>
However, both of these equations are such that $\Delta = 0$ and have one unique solution.<br>
<br>
The reason why the algorithm doesn't return the current answer is because the variable `delta` is rarely equal to 0 in value. Due to `float` approximations, it is very close to 0 but not equal to it.<br>
The solution to tackle this issue is to define a very small value $\varepsilon$ and consider that the discriminant $\Delta$ is equal to 0 when it is very small compared to $b^2$ : $\vert \Delta \vert \leq \varepsilon b^2$.<br>
<br>
For $\varepsilon = 2^{-52}$, our algorithm becomes :

In [37]:
from numpy import sqrt

def solve(a, b, c, epsilon = 2**(-52)):
    '''
    Parameters : floats a, b, c, float epsilon
    Result : solution(s) of the equation ax^2 + bx + c = 0
    '''
    delta = b**2 - 4*a*c
    print('Delta :', delta)
    if delta < - epsilon * b**2:
        print('No real solutions')
    elif delta > epsilon * b**2:
        x1, x2 = (-b-sqrt(delta))/(2*a), (-b+sqrt(delta))/(2*a)
        print('Two solutions :', x1, 'and', x2)
    else:
        x = -b/(2*a)
        print('One solution :', x)

solve(0.01, 0.2, 1)
solve(0.011025, 0.21, 1)

Delta : 6.938893903907228e-18
One solution : -10.0
Delta : -6.938893903907228e-18
One solution : -9.523809523809524


This value of $\varepsilon = 2^{-52}$ isn't chosen randomy : it is actually the **machine epsilon**. It is an upper bound of the error due to rounding `float` : it is such that $x - \varepsilon \vert x \vert$ and $x + \varepsilon \vert x \vert$ are indistinguishable by the machine.<br>
Taking a smaller value for the variable `epsilon` will result in the same issue as what we had in the first function :

In [38]:
solve(0.01, 0.2, 1, epsilon = 2**(-53))
solve(0.011025, 0.21, 1, epsilon = 2**(-53))

Delta : 6.938893903907228e-18
Two solutions : -10.000000131708903 and -9.999999868291098
Delta : -6.938893903907228e-18
No real solutions


From now on, when writing algorithms to find solutions to equations, we will only look for approximate solutions as we are limited by the machine's precision.

## 2) Bisection method

The **bisection method** is an algorithm to **find roots of functions** which means finding the solution of an equation of the form $f(x) = 0$ where $f$ is a continuous function.<br>
The fact that $f$ is continuous is important because this method relies on the **itermediate value theorem** stating, in particular, that if $a < b$ are two real numbers such that $f(a)$ and $f(b)$ are of different signs, there exists $x \in [a,b]$ such that $f(x) = 0$.<br>
The condition that $f(a)$ and $f(b)$ are of different signs can be written $f(a)f(b) < 0$, which is the condition we will consider in our Python algorithm.<br>
<br>
The key idea of the bisection method is to separate $[a,b]$ into two sub-segments $[a,m]$ and $[m,b]$ where $m = (a+b)/2$ is the middle of the segment $[a,b]$. There are now two possibilities :

- If $f(a)f(m) < 0$ : This means that the root is inside the segment $[a,m]$ and we can restrict our search there.
- If $f(a)f(m) \geq 0$ : This means that the root is inside the segment $[m,b]$ and we can restrict our search there.

We keep splitting the new segment again and repeat the process until we are close enough to the root, which means when the segment is of length inferior to the precision $\varepsilon$ we want.<br>
<br>
Note that this algorithm only works if the is a **unique root inside the segment $[a,b]$** !

In [62]:
def bisect(f, a, b, epsilon = 1e-9):
    '''
    Parameter : function f, floats a, b, float epsilon
    Result : zero of the function f in the segment [a,b]
    '''
    while b - a > 2 * epsilon:
        m = (a+b) / 2
        if f(a)*f(m) < 0:
            a, b = a, m
        else:
            a, b = m, b
    return (a + b) / 2

Let's try using this algorithm to approximate the positive solution of the equation $e^x = x^2 + 2$. Looking at the graph of $y = e^x - x^2 - 2$, we notice that the solution is in the interval $[0,2]$ : we can pick $a = 0$ and $b = 2$

In [65]:
from math import exp

print(bisect(lambda x : exp(x) - x**2 - 2, 0, 2))
print(bisect(lambda x : exp(x) - x**2 - 2, 0, 2, 1e-15))

1.3190736761316657
1.3190736768573652


There is actually a Python library, named `scipy` that is dedicated to numerical calculations. A similar function using the bisection method to approximate the zero of a function can be found under the name `bisect` in the library `scipy.optimize`.<br>
The only difference is that it returns an error when exceeding a certain maximum iteration amount `max_iter` in the while loop to ensure the algorithm doesn't loop endlessly. We can change our own `bisect` function to include this behavior too !

In [66]:
def bisect(f, a, b, epsilon = 1e-9, max_iter = 100):
    '''
    Parameter : function f, floats a, b, float epsilon
    Result : zero of the function f in the segment [a,b]
    '''
    n_iter = 0
    while b - a > 2 * epsilon:
        n_iter += 1
        if n_iter > max_iter:
            message = 'Failure after ' + str(max_iter) + ' iterations.'
            raise RunTimeError(message)
        m = (a + b) / 2
        if f(a)*f(m) < 0:
            a, b = a, m
        else:
            a, b = m, b
    return (a + b) / 2

Now, the function cannot loop forever, even if we take a value $\varepsilon$ that is too small for the algorithm to distinguish (a+b)/2 from a or b

## 3) Rectangle method

The **rectangle method** is an algorithm to approximate integrals. For a continuous function $f$ over a segment $[a,b]$, the Riemann integral of $f$ over $[a,b]$ is defined by
$$\int_{a}^{b}f(x)dx := \lim_{N → +\infty} \sum_{k=0}^{N-1} \frac{b-a}{N} f\left(a + \frac{k(b-a)}{N}\right).$$

Therefore, we can approximate integrals by calculating the sum of the right hand side for big values of $N$ :
$$\int_{a}^{b}f(x)dx \approx \sum_{k=0}^{N-1} \frac{b-a}{N} f\left(a + \frac{k(b-a)}{N}\right).$$

It is then very easy to implement this method in Python :

In [96]:
def rectangle(f, a, b, N):
    '''
    Parameters : function f, floats a, b, int N
    Result : approximation of the integral of f over [a,b] with a sum of N rectangles
    '''
    res = 0
    dx = (b - a) / N
    for k in range(N):
        x = a + k*(b-a)/N
        res += f(x) * dx
    return res

We'll use this algorithm to approximate $\pi$ with the following integral, representing the area of a semi-circle of radius 1 :
$$ \int_{-1}^{1} \sqrt{1-x^2} dx = \frac{\pi}{2}$$

In [97]:
from math import sqrt

N = 1000
print('Rectangle method for N = 1 000 :', 2 * rectangle(lambda x : sqrt(1 - x**2), -1, 1, N))

N = 1000000
print('Rectangle method for N = 1 000 000 :', 2 * rectangle(lambda x : sqrt(1 - x**2), -1, 1, N))

Rectangle method for N = 1 000 : 3.141487477002142
Rectangle method for N = 1 000 000 : 3.1415926502634544


As we can see, for N = 1 000 000, we already obtain 8 correct decimals of $\pi$ !<br>
<br>
We can easily improve this method by "centering" the rectangles used in the Riemann sum :
$$\int_{a}^{b}f(x)dx \approx \sum_{k=0}^{N-1} \frac{b-a}{N} f\left(a + \frac{(k+1/2)(b-a)}{N}\right).$$

In [100]:
def rectangle_centered(f, a, b, N):
    '''
    Parameters : function f, floats a, b, int N
    Result : approximation of the integral of f over [a,b] with a sum of N rectangles
    '''
    res = 0
    dx = (b - a) / N
    for k in range(N):
        x = a + (k+1/2)*(b-a)/N
        res += f(x) * dx
    return res

from math import sqrt

N = 1000
print('Centered rectangle method for N = 1 000 :', 2 * rectangle_centered(lambda x : sqrt(1 - x**2), -1, 1, N))

N = 1000000
print('Centered rectangle method for N = 1 000 000 :', 2 * rectangle_centered(lambda x : sqrt(1 - x**2), -1, 1, N))

Centered rectangle method for N = 1 000 : 3.1416234568199135
Centered rectangle method for N = 1 000 000 : 3.1415926545640898


As we can see, it's even better than the first rectangle method ! There exists other method to approximate integrals by selecting other points and using polynomial curves instead or simply straight lines to fit the curve of $f$ but we won't cover these methods.